<a href="https://colab.research.google.com/github/Saumya0330/Saumya-USC420/blob/main/sine_wave.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile sine_wave.cu
#include <stdio.h>
#include <math.h>

#define Pi 3.14159265358979323846264338327950

__global__ void sine_wave(double *x, double *y, double amplitude, double frequency, double step, int num_points)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if(idx<num_points){
    x[idx] = step * idx;
    y[idx] = amplitude * sin( frequency * x[idx]);

    }
}
int main()
{
  double *h_x, *h_y;
  double *d_x, *d_y;
  int num_points=1440;
  double step=0.00436332;
  double amplitude=1;
  double frequency=1;
  int size=num_points*sizeof(double);
  h_x=(double*)malloc(size);
  h_y=(double*)malloc(size);

  cudaMalloc((void**)&d_x,size);
  cudaMalloc((void**)&d_y,size);
  int blockSize = 256;
    int gridSize = (num_points + blockSize - 1) / blockSize;
  sine_wave<<<gridSize,blockSize>>>(d_x,d_y,amplitude,frequency,step,num_points);
  cudaDeviceSynchronize();

  cudaMemcpy(h_x,d_x,size,cudaMemcpyDeviceToHost);
  cudaMemcpy(h_y,d_y,size,cudaMemcpyDeviceToHost);

  printf("x (radians)\ty (sine)\n");
    for (int i = 0; i < num_points; i++) {
        printf("%f\t%f\n", h_x[i], h_y[i]);
    }

    FILE *gp = popen("gnuplot -persistent", "w");
    if (gp == NULL) {
        fprintf(stderr, "Error opening pipe to gnuplot.\n");
        return 1;
    }

    // Configure gnuplot
    fprintf(gp, "set title 'Sine Wave'\n");
    fprintf(gp, "set xlabel 'x (radians)'\n");
    fprintf(gp, "set ylabel 'y (sine)'\n");
    fprintf(gp, "plot '-' with lines title 'A*sin(f*x)'\n");
int i;
    for (i = 0; i < num_points; i++) {
        fprintf(gp, "%f %f\n", h_x[i], h_y[i]);
    }
    fprintf(gp, "e\n");  // End of data input for gnuplot
    fflush(gp);           // Flush the data to gnuplot

    // Close the gnuplot pipe
    pclose(gp);

  cudaFree(d_x);
  cudaFree(d_y);
  free(h_x);
  free(h_y);
  return 0;
}

Writing sine_wave.cu


In [ ]:
!nvcc sine_wave.cu -o sine_wave

In [ ]:
!./sine_wave

x (radians)	y (sine)
0.000000	0.000000
0.004363	0.004363
0.008727	0.008727
0.013090	0.013090
0.017453	0.017452
0.021817	0.021815
0.026180	0.026177
0.030543	0.030538
0.034907	0.034899
0.039270	0.039260
0.043633	0.043619
0.047997	0.047978
0.052360	0.052336
0.056723	0.056693
0.061086	0.061048
0.065450	0.065403
0.069813	0.069756
0.074176	0.074108
0.078540	0.078459
0.082903	0.082808
0.087266	0.087156
0.091630	0.091502
0.095993	0.095846
0.100356	0.100188
0.104720	0.104528
0.109083	0.108867
0.113446	0.113203
0.117810	0.117537
0.122173	0.121869
0.126536	0.126199
0.130900	0.130526
0.135263	0.134851
0.139626	0.139173
0.143990	0.143493
0.148353	0.147809
0.152716	0.152123
0.157080	0.156434
0.161443	0.160742
0.165806	0.165047
0.170169	0.169349
0.174533	0.173648
0.178896	0.177943
0.183259	0.182235
0.187623	0.186524
0.191986	0.190809
0.196349	0.195090
0.200713	0.199368
0.205076	0.203642
0.209439	0.207912
0.213803	0.212178
0.218166	0.216439
0.222529	0.220697
0.226893	0.224951
0.231256	0.229200
0.23561

In [ ]:
%%writefile sine_wave_fft.cu
#include <stdio.h>
#include <math.h>
#include <cufft.h>

#define N 1024 // Number of points
#define PI 3.1415926535897932

// Corrected Kernel: Generate Sine Wave
__global__ void generateSineWave(float* signal, int n, float freq, float sampleRate) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    if (i < n) {
        signal[i] = sinf(2 * PI * freq * i / sampleRate);
    }
}

int main() {
    // Parameters for sine wave
    float freq = 50.0f;        // Frequency in Hz
    float sampleRate = 1000.0f;  // Sampling rate in Hz

    // Host and Device memory allocation
    float *d_signal;
    cufftComplex *d_fftResult;

    cudaMalloc((void**)&d_signal, N * sizeof(float));
    cudaMalloc((void**)&d_fftResult, (N / 2 + 1) * sizeof(cufftComplex));

    // Generate sine wave on GPU
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;
    generateSineWave<<<numBlocks, blockSize>>>(d_signal, 1024, freq, sampleRate);

    // CUFFT plan and execution
    cufftHandle plan;
    cufftPlan1d(&plan, N, CUFFT_R2C, 1);
    cufftExecR2C(plan, d_signal, d_fftResult);

    // Retrieve the results (optional, for verification)
    cufftComplex* h_fftResult = (cufftComplex*)malloc((N / 2 + 1) * sizeof(cufftComplex));
    cudaMemcpy(h_fftResult, d_fftResult, (N / 2 + 1) * sizeof(cufftComplex), cudaMemcpyDeviceToHost);

    // Print first few frequency domain values
    for (int i = 0; i < 10; i++) {
        printf("Frequency bin %d: %f + %fi\n", i, h_fftResult[i].x, h_fftResult[i].y);
    }

    // Cleanup
    cufftDestroy(plan);
   cudaFree(d_signal);
    cudaFree(d_fftResult);
    free(h_fftResult);

    return 0;
}


Overwriting sine_wave_fft.cu


In [ ]:
!nvcc sine_wave_fft.cu -o sine_wave_fft -lcufft
!./sine_wave_fft

Frequency bin 0: 1.705802 + 0.000000i
Frequency bin 1: 1.706656 + -0.059626i
Frequency bin 2: 1.709188 + -0.119403i
Frequency bin 3: 1.713400 + -0.179459i
Frequency bin 4: 1.719297 + -0.239931i
Frequency bin 5: 1.726959 + -0.300908i
Frequency bin 6: 1.736432 + -0.362641i
Frequency bin 7: 1.747695 + -0.425190i
Frequency bin 8: 1.760848 + -0.488780i
Frequency bin 9: 1.775920 + -0.553499i


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!ls /usr/local/cuda/lib64 | grep cufft

libcufft.so
libcufft.so.11
libcufft.so.11.0.8.103
libcufft_static.a
libcufft_static_nocallback.a
libcufftw.so
libcufftw.so.11
libcufftw.so.11.0.8.103
libcufftw_static.a
